In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("online_retail.csv")

In [3]:
df.head()

,index,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [4]:
df.columns

Index(['index', 'InvoiceNo', 'StockCode', 'Description', 'Quantity',
       'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

###### Data cleaning

In [5]:
# Removing cancellations
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

# Dropping missing customers
df = df.dropna(subset=['CustomerID'])

# Converting types
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['Revenue'] = df['Quantity'] * df['UnitPrice']

######  RFM Feature Engineering

In [6]:
snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'nunique',
    'Revenue': 'sum'
}).reset_index()

rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']


In [8]:
rfm.head()

,CustomerID,Recency,Frequency,Monetary
0,12346.0,326,1,77183.60
1,12347.0,2,7,4310.00
2,12348.0,75,4,1797.24
3,12349.0,19,1,1757.55
4,12350.0,310,1,334.40


###### CLV modeling

In [9]:
from sklearn.ensemble import RandomForestRegressor

X = rfm[['Recency', 'Frequency', 'Monetary']]
y = rfm['Monetary'] 

model = RandomForestRegressor(random_state=42)
model.fit(X, y)

rfm['Predicted_CLV'] = model.predict(X)

######  Customer Segmentation

In [10]:
rfm['CLV_Segment'] = pd.qcut(
    rfm['Predicted_CLV'],
    q=3,
    labels=['Low', 'Medium', 'High']
)

In [11]:
rfm.head()

,CustomerID,Recency,Frequency,Monetary,Predicted_CLV,CLV_Segment
0,12346.0,326,1,77183.60,75860.0654,High
1,12347.0,2,7,4310.00,4309.3891,High
2,12348.0,75,4,1797.24,1797.2365,High
3,12349.0,19,1,1757.55,1758.5355,High
4,12350.0,310,1,334.40,334.4234,Low


In [12]:
# Saving the dataset
final_table = rfm.merge(
    df[['CustomerID', 'Country']].drop_duplicates(),
    on='CustomerID',
    how='left'
)

final_table.to_csv("clv_looker_data.csv", index=False)


In [13]:
df2 = pd.read_csv("clv_looker_data.csv")
df2.head()

,CustomerID,Recency,Frequency,Monetary,Predicted_CLV,CLV_Segment,Country
0,12346.0,326,1,77183.60,75860.0654,High,United Kingdom
1,12347.0,2,7,4310.00,4309.3891,High,Iceland
2,12348.0,75,4,1797.24,1797.2365,High,Finland
3,12349.0,19,1,1757.55,1758.5355,High,Italy
4,12350.0,310,1,334.40,334.4234,Low,Norway


In [14]:
df2.columns

Index(['CustomerID', 'Recency', 'Frequency', 'Monetary', 'Predicted_CLV',
       'CLV_Segment', 'Country'],
      dtype='object')